# **Importar libreria**

In [ ]:
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pydicom as dicom
import cv2
import ast

import glob
import re
import math
from tqdm.notebook import tqdm
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from pydicom.pixel_data_handlers.util import apply_voi_lut
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import applications 

In [ ]:
#Enlace de la base de datos
path = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/'
os.listdir(path)

# **Cargar base de datos**

In [ ]:
train_data = pd.read_csv(path+'train_labels.csv')
samp_subm  = pd.read_csv(path+'sample_submission.csv')

print(train_data.head(7)),print(samp_subm.head())

# **2 Compresión de los datos** 
Metodologia Crisp-DM

In [ ]:
#Imprimir tamaño de carpetas por TRAIN Y TEST
print('Samples train:', len(train_data))
print('Samples test:', len(samp_subm))

In [ ]:
train_data.head(7)

In [ ]:
## analisis de datos faltantes
print(pd.isnull(train_data).sum()) 
print('___________')
print(pd.isnull(samp_subm ).sum())

In [ ]:
train_data["MGMT_value"].value_counts()

In [ ]:
to_exclude = [109, 123, 709]

train_data = train_data[~train_data['BraTS21ID'].isin(to_exclude)]
num_samples = train_data.shape[0]
num_positives = np.sum(train_data['MGMT_value'] == 1)
num_negatives = np.sum(train_data['MGMT_value'] == 0)


train_data["MGMT_value"].value_counts().head(2).plot(kind = 'pie',autopct='%1.1f%%', figsize=(8, 8)).legend()


train_data.hist(column="MGMT_value")

In [ ]:
samp_subm.head()

In [ ]:
#analizar una carpeta 100--->00150
folder = str(train_data.loc[100, 'BraTS21ID']).zfill(5)
folder

In [ ]:
## CONTENIDO DE LAS CARPETAS
os.listdir(path+'train/'+folder)

Explicacion de las tecnicas T2w, T1wCE, T1w, FLAIR
![](https://i.postimg.cc/3NS5z6RB/image.png)


In [ ]:
#numero de imagenes por capeta
print('Number of FLAIR images:', len(os.listdir(path+'train/'+folder+'/'+'FLAIR')))
print('Number of T1w images:',   len(os.listdir(path+'train/'+folder+'/'+'T1w')))
print('Number of T1wCE images:', len(os.listdir(path+'train/'+folder+'/'+'T1wCE')))
print('Number of T2w images:',   len(os.listdir(path+'train/'+folder+'/'+'T2w')))

In [ ]:
#Cargar imagenes
path_file = ''.join([path, 'train/', folder, '/', 'FLAIR/'])
image = os.listdir(path_file)[0]
data_file = dicom.dcmread(path_file+image)
img = data_file.pixel_array

In [ ]:
#Forma de la imagen (Tamaño)
print('Image shape:', img.shape)
###print(train_data.shape)

In [ ]:
## IMAGENES FLAIR
def plot_examples(row = 0, cat = 'FLAIR'): 
    folder = str(train_data.loc[row, 'BraTS21ID']).zfill(5)
    path_file = ''.join([path, 'train/', folder, '/', cat, '/'])
    images = os.listdir(path_file)
    
    fig, axs = plt.subplots(1, 5, figsize=(30, 30))
    fig.subplots_adjust(hspace = .2, wspace=.2)
    axs = axs.ravel()
    
    for num in range(5):
        data_file = dicom.dcmread(path_file+images[num])
        img = data_file.pixel_array
        axs[num].imshow(img, cmap='gray')
        axs[num].set_title(cat+' '+images[num])
        axs[num].set_xticklabels([])
        axs[num].set_yticklabels([])
        
row = 0
plot_examples(row = row, cat = 'FLAIR')

In [ ]:
##IMAGENES T1W
plot_examples(row = row, cat = 'T1w')

In [ ]:
## IMAGENES T1CE
plot_examples(row = row, cat = 'T1wCE')

In [ ]:
##IMAGENES T2W
plot_examples(row = row, cat = 'T2w')

![](https://i.pinimg.com/originals/15/7f/99/157f9957ecd64da6b079ff5189dbf3ff.gif)

# **3 Preparacion de los datos**

PREPROCESAMIENTO DE IMAGEN

Para cada paciente, realizaremos un preprocesado de las imágenes aplicando estas diferentes modificaciones:
* Cargue una secuencia ordenada de 64 resonancias magnéticas
* Recortar imágenes para reducir los bordes negros
* Cambiar el tamaño de la imagen para reducir los 0 de la matriz para el modelo previo al entrenamiento
* Aplicar filtro de eliminación de ruido
* Convierta cada imagen en una matriz 3D


In [ ]:
#Datos
data_directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'
train_df = pd.read_csv(data_directory+"train_labels.csv")
train_df['BraTS21ID5'] = [format(x, '05d') for x in train_df.BraTS21ID]
train_df.head(3)
test = pd.read_csv(
    data_directory+'sample_submission.csv')

test['BraTS21ID5'] = [format(x, '05d') for x in test.BraTS21ID]

In [ ]:
IMAGE_SIZE = 240
SCALE = .8
NUM_IMAGES = 64
MRI_TYPE = "FLAIR"

In [ ]:
#CARGAR IMAEGN dicom, RECORTAR Y APLICAR FILTRO DENOISING 
def load_dicom_image(
    path,
    img_size=IMAGE_SIZE,
    scale=SCALE):
# Cargar imagen 
    img = dicom.read_file(path).pixel_array
    # recorte imagen
    center_x, center_y = img.shape[1] / 2, img.shape[0] / 2
    width_scaled, height_scaled = img.shape[1] * scale, img.shape[0] * scale
    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
    img = img[int(top_y):int(bottom_y), int(left_x):int(right_x)]

    img = cv2.resize(img, (img_size, img_size))
    #cv2.fastNlMeansDenoisingMulti() 
    
    # Convertir en matriz 3D
    img = np.repeat(img[..., np.newaxis], 3, -1)
    
    return img

![](http://people.tuebingen.mpg.de/burger/neural_denoising/images/denoising.png)

Podemos verificar el resultado de estos diferentes pasos de preprocesamiento

In [ ]:
sample_img = dicom.read_file(
    data_directory+"train/00046/FLAIR/Image-115.dcm").pixel_array
preproc_img = load_dicom_image(data_directory+"train/00046/FLAIR/Image-115.dcm")


fig = plt.figure(figsize=(12,8))
ax1 = plt.subplot(1,2,1)
ax1.imshow(sample_img, cmap="gray")
ax1.set_title(f"Original image shape = {sample_img.shape}")
ax2 = plt.subplot(1,2,2)
ax2.imshow(preproc_img[:,:,0], cmap="gray")
ax2.set_title(f"Preproc image shape = {preproc_img.shape}")
plt.show()

Selecionamos imagen central de cada carpeta y selecionamos igual numero de imaganes a cada lado (Evitar imagenes negras)

In [ ]:
def load_dicom_images_3d(
    scan_id, 
    num_imgs=NUM_IMAGES, 
    img_size=IMAGE_SIZE, 
    mri_type=MRI_TYPE, 
    split="train"):
    
    files = sorted(glob.glob(f"{data_directory}{split}/{scan_id}/{mri_type}/*.dcm"), 
               key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])

    middle = len(files)//2
    num_imgs2 = num_imgs//2
    p1 = max(0, middle - num_imgs2)
    p2 = min(len(files), middle + num_imgs2)
    img3d = np.stack([load_dicom_image(f) for f in files[p1:p2]]) 
    if img3d.shape[0] < num_imgs:
        n_zero = np.zeros((num_imgs - img3d.shape[0], img_size, img_size, 3))
        img3d = np.concatenate((img3d,  n_zero), axis = 0)
            
    return img3d

Prueba de carga de una secuencia de imágenes preprocesadas para un paciente:

In [ ]:
sample_seq = load_dicom_images_3d("00046")
print("Shape of the sequence is:", sample_seq.shape)
print("Dimension of the 15th image in sequence is:", sample_seq[15].shape)
fig = plt.figure(figsize=(5,5))
plt.imshow(np.squeeze(sample_seq[15][:,:,0]), cmap="gray")
plt.show()

# **4 Modelado**

CARGAR MODELO RESNET50 PREENTRENADO
Para realizar el Transfer Learning sobre cada imagen de la secuencia, cargaremos un modelo preentrenado gracias a Keras.applications con los pesos preentrenados en ImageNet.

![](https://i0.wp.com/www.datasmarts.net/wp-content/uploads/2019/10/1_IlzW43-NtJrwqtt5Xy3ISA.jpeg?fit=750%2C300&ssl=1)

In [ ]:
base_resnet = keras.applications.ResNet50(
    weights=None,
    #weights="imagenet",
    pooling='avg',
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    include_top=False)

"""
base_resnet.save_weights(
'base_resnet_imagenet.h5')

"""
#base_resnet.load_weights(
# '../input/resnet-imagenet-weights/base_resnet_imagenet.h5')


También vamos a corregir todas las capas del modelo para que no se vuelvan a entrenar para la detección de características. La capa de clasificación tampoco se carga (include_top = False).

In [ ]:
base_resnet.trainable = False

* Crear una matriz en base a RESNET50 para cada secuencia de paciente
* modelo ResNet50 para obtener los pesos segun la predicción de cada imagen de los pacientes.
* Creamos una matriz global que agrupará las secuencias de x matrices ResNet50 para todos los pacientes.

In [ ]:
train = train_df[['BraTS21ID5','MGMT_value']]
X_train = train['BraTS21ID5'].values
y_train = train['MGMT_value'].values

In [ ]:
listMatrix = []
for i, patient in enumerate(tqdm(X_train)):
    listVectors = []
    sequence = load_dicom_images_3d(scan_id=str(patient),mri_type=MRI_TYPE)
    for j in range(len(sequence)):
        img = sequence[j]
        img = np.expand_dims(img, axis=0)
        img = tf.keras.applications.resnet50.preprocess_input(img)
        img_vector = base_resnet.predict(img)
        listVectors.append(np.array(img_vector))
    
    PatientMatrix = np.stack(listVectors)
    listMatrix.append(PatientMatrix)

Veamos ahora las formas de las matrices obtenidas tras la aplicación de este Learning Transfer:

In [ ]:
print(f"Number of patient matrix: {len(listMatrix)}")
print(f"Patient matrix shape: {listMatrix[0].shape}")

![](https://aprendeconalf.es/docencia/python/manual/img/arrays.png)

In [ ]:
#np.array(listMatrix, dtype=object).shape

aplicar LSTM para la clasificacion
Las redes neuronales recurrentes (RNN) son ampliamente utilizadas en inteligencia artificial cuando una noción temporal está involucrada en los datos.


In [ ]:
model_input_dim = listMatrix[0].shape[2]

In [ ]:
def get_sequence_model():
    '''Definicion de la arquitectura LSTM '''
    model = keras.models.Sequential()
    model.add(keras.layers.LSTM(100, input_shape=(NUM_IMAGES, model_input_dim), return_sequences=True))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(100, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    return model

![](https://upload.wikimedia.org/wikipedia/commons/f/f2/K-fold_cross_validation.jpg)

# Explicacion K-FOLD  ↓

In [ ]:
from sklearn.model_selection import KFold

inputs = np.array(listMatrix)
targets = np.array(y_train).astype('float32').reshape((-1,1))

num_folds = 5

# Definir la validación cruzada de K-fold
kfold = KFold(n_splits=num_folds, shuffle=True)

# Evaluación del modelo K-fold Cross Validation
history = {}
fold_no = 1
for train_df, valid_df in kfold.split(inputs, targets):
    
    train_dataset = tf.data.Dataset.from_tensor_slices((inputs[train_df], targets[train_df]))
    valid_dataset = tf.data.Dataset.from_tensor_slices((inputs[valid_df], targets[valid_df]))
    
    model = get_sequence_model()
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics='accuracy')
    
    # Define callbacks.
    model_save = ModelCheckpoint(f'Brain_lstm_kfold_{fold_no}.h5', 
                                 save_best_only = True, 
                                 monitor = 'val_accuracy', 
                                 mode = 'max', verbose = 1)
    early_stop = EarlyStopping(monitor = 'val_accuracy', 
                               patience = 25, mode = 'max', verbose = 1,
                               restore_best_weights = True)
    
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    epochs = 200
    history[fold_no] = model.fit(
        train_dataset,
        validation_data=valid_dataset, 
        epochs=epochs, 
        batch_size=32,
        callbacks = [model_save, early_stop])
    
    # Aumentar el número de pliegues
    fold_no += 1

Ahora entrenaremos este modelo LSTM en las matrices compiladas para cada paciente utilizando Transfer Learning ResNet50.

Se configura un EarlyStopping y se guardará el mejor modelo.

Ahora veamos los resultados de este entrenamiento:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 7))
ax = ax.ravel()

for fold in history:
    for i, metric in enumerate(["accuracy","loss"]):
        ax[i].plot(history[fold].history[metric], label="train "+str(fold))
        ax[i].plot(history[fold].history["val_" + metric], linestyle="dotted", label="val "+str(fold))
        ax[i].set_title("Model {}".format(metric))
        ax[i].set_xlabel("epochs")
        ax[i].set_ylabel(metric)
        ax[i].legend()